In [4]:
import sys
sys.path.append("../src")
from mistral_client import run_mistral
from ner_post_processing import parse_entities_promptner, get_token_labels

import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
from collections import defaultdict

/home/dachun/miniconda3/envs/mistral-ai-hackathon/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset = load_dataset("DFKI-SLT/cross_ner", "politics", split="test")

In [6]:
[" ".join(tokens) for tokens in dataset["tokens"]]

['They mostly engage in confronting the police during demonstrations and riots , especially in countries like Canada , Mexico or Greece .',
 'Lincoln replaced Buell with William Rosecrans ; and after the 1862 and 1863 United States House of Representatives elections he replaced McClellan with Ambrose Burnside .',
 'Tamsin Greig narrated , and the cast included Nicky Henson as Napoleon , Toby Jones as the propagandist Squealer , and Ralph Ineson as Boxer .',
 'Many of the most prominent national leaders , such as Washington , John Adams , John Hancock , and Benjamin Franklin , retired from public life , served as foreign delegates , or held office in state governments ; and for the general public , local government and self-rule seemed quite satisfactory .',
 'Einstein became a full professor at the German Charles-Ferdinand University in Prague in April 1911 , accepting Austria n citizenship in the Austria-Hungary to do so .',
 'Locker-Lampson took Einstein to meet Winston Churchill at 

In [7]:
dataset

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 651
})

In [5]:
fp = "../data/scored/politics_tasklevel_french_kw.old.csv"

In [6]:
df = pd.read_csv(fp)

In [7]:
import ast

df = df[df["ner_tags"].apply(lambda x: 79 in ast.literal_eval(x) or 81 in ast.literal_eval(x))]

In [8]:
df.to_csv("../data/scored/politics_tasklevel_french_kw.filt.csv", index=False)

In [9]:
len(df)

333

In [31]:
fp_french = "../data/scored/politics_tasklevel_french_kw.csv"
fp_french_v2 = "../data/scored/politics_tasklevel_french_kw_v2.csv"
fp_french_knn = "../data/scored/french_knn.csv"

In [32]:
df_french = pd.read_csv(fp_french)
df_french_v2 = pd.read_csv(fp_french_v2)
df_french_knn = pd.read_csv(fp_french_knn)

In [33]:
df_comb = pd.concat([df_french, df_french_v2, df_french_knn])
df_comb = df_comb.drop_duplicates(subset="prompt")

In [34]:
df_comb

,tokens,prompt,output,ner_tags
0,"['On', '22', 'July', '2014', ',', 'King', 'Phi...","\nDfn: An entity is a person (person), organis...","1. 22 July 2014 | False | as it is a date, not...","[0, 0, 0, 0, 0, 51, 52, 0, 79, 80, 0, 81, 0, 0..."
1,"['More', 'recently', ',', 'the', 'memory', 'of...","\nDfn: An entity is a person (person), organis...",1. Charles Martel | True | as he is a French p...,"[0, 0, 0, 0, 0, 0, 79, 80, 0, 0, 0, 0, 0, 0, 0..."
2,"['In', 'early', '2013', ',', 'documents', 'pro...","\nDfn: An entity is a person (person), organis...",1. early 2013 | False | as it is a time period...,"[0, 0, 0, 0, 0, 0, 0, 51, 52, 0, 0, 0, 0, 0, 0..."
3,"['Before', 'this', ',', 'three', 'territories'...","\nDfn: An entity is a person (person), organis...",1. Three territories | False | as it is a vagu...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 41, 0, 0, 0,..."
4,"['Distrust', 'of', 'national', 'government', '...","\nDfn: An entity is a person (person), organis...",1. Distrust of national government | False | a...,"[0, 0, 0, 0, 0, 0, 0, 21, 0, 0, 0, 0, 0, 21, 0..."
...,...,...,...,...
676,"['""', 'In', '1947', ',', 'Saud', 'visited', 't...","\nDfn: An entity is a person (person), organis...","1. 1947 | False | as it is a date, not an enti...","[0, 0, 0, 0, 51, 0, 0, 21, 22, 0, 0, 0, 51, 52..."
678,"['In', '1964', ',', 'Castro', 'was', 'selected...","\nDfn: An entity is a person (person), organis...","1. 1964 | False | as it is a date, not an enti...","[0, 0, 0, 51, 0, 0, 0, 51, 52, 52, 52, 0, 0, 2..."
680,"['The', 'CPGB', '(', 'PCC', ')', 'endorsed', '...","\nDfn: An entity is a person (person), organis...",1. CPGB (PCC) | True | as it is a political pa...,"[0, 41, 42, 42, 42, 0, 0, 41, 42, 0, 0, 25, 26..."
681,"['Hollande', 'was', 'widely', 'blamed', 'for',...","\nDfn: An entity is a person (person), organis...",1. Hollande | True | as he is a French politic...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25,..."


In [35]:
df_comb.to_csv("../data/scored/french_politics.comb.csv", index=False)

In [36]:
fp_uniform1000 = "../data/scored/politics_tasklevel_uniform1000.csv"
fp_uniform10000 = "../data/scored/politics_tasklevel_uniform10000.csv"

In [37]:
df_1000 = pd.read_csv(fp_uniform1000)
df_10000 = pd.read_csv(fp_uniform10000)

df_comb = pd.concat([df_1000, df_10000])
df_comb = df_comb.drop_duplicates(subset="prompt")

In [39]:
df_comb.to_csv("../data/scored/politics_tasklevel_uniform.comb.csv", index=False)

In [25]:
entities = []
for row in df_comb["output"]:
    row_entities = parse_entities_promptner(row)
    entities += [entity for entity in row_entities if entity[3] == "frenchpolitician"]

# get the frequency of each entity
entity_freq = defaultdict(int)
for entity in entities:
    entity_freq[entity[0]] += 1

In [28]:
# print the counter sorted by value
print(sorted(entity_freq.items(), key=lambda x: x[1], reverse=True))
# get all entities with frequency > 3
entities = [entity for entity in entities if entity_freq[entity[0]] > 3]

[('Emmanuel Macron', 40), ('Nicolas Sarkozy', 35), ('Marine Le Pen', 23), ('François Hollande', 22), ('Jean-Marie Le Pen', 19), ('Le Pen', 17), ('François Bayrou', 14), ('Hollande', 13), ('Jacques Chirac', 13), ('Chirac', 13), ('Sarkozy', 10), ('Jean-Luc Mélenchon', 9), ('Hamon', 9), ('President François Hollande', 8), ('Macron', 8), ('French President François Hollande', 8), ('François Fillon', 8), ('Jospin', 7), ('French President Emmanuel Macron', 7), ('Bayrou', 7), ('Laurent Fabius', 7), ('Édouard Philippe', 6), ('Mélenchon', 6), ('Dominique de Villepin', 6), ('Benoît Hamon', 6), ('Mitterrand', 6), ('Jean-Marc Ayrault', 5), ('Manuel Valls', 5), ('Ségolène Royal', 5), ('François Mitterrand', 5), ('President Emmanuel Macron', 4), ('Alain Juppé', 4), ('Dominique Strauss-Kahn', 4), ('Jean-Louis Borloo', 4), ('Jean-Yves Le Drian', 4), ('President Jacques Chirac', 4), ('Douste-Blazy', 4), ('Charles de Gaulle', 4), ('Gilbert Collard', 3), ('Marion Maréchal', 3), ('He', 3), ('Juppé', 3), (

In [30]:
# print the entities as a list of just the entity name
list(set([entity[0].lower() for entity in entities]))

['french president françois hollande',
 'mélenchon',
 'jean-marc ayrault',
 'jean-louis borloo',
 'president emmanuel macron',
 'charles de gaulle',
 'nicolas sarkozy',
 'hamon',
 'jospin',
 'jacques chirac',
 'ségolène royal',
 'chirac',
 'president françois hollande',
 'bayrou',
 'jean-yves le drian',
 'jean-luc mélenchon',
 'françois hollande',
 'françois fillon',
 'alain juppé',
 'laurent fabius',
 'emmanuel macron',
 'françois bayrou',
 'françois mitterrand',
 'sarkozy',
 'dominique de villepin',
 'jean-marie le pen',
 'le pen',
 'manuel valls',
 'dominique strauss-kahn',
 'douste-blazy',
 'french president emmanuel macron',
 'marine le pen',
 'hollande',
 'benoît hamon',
 'mitterrand',
 'president jacques chirac',
 'macron',
 'édouard philippe']